In [37]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [49]:
mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils

In [3]:
#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))

In [4]:
# extract keypoints of the features in image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

In [2]:
os.path.join("Sign_Images")

'Sign_Images'

In [38]:
#path for flatten datas
dataPath = os.path.join('Sign_Data')

#path for images
imgPath= os.path.join('Sign_Image')

signs=np.array(["Ambulance","Fight", "Clap", "Hungry", "Help", "Medicine","Accident","ill","Hospital","Washroom"])

#90 video of each sign
numSequences=90

#length of each video
sequenceLength=30

In [39]:
for sign in signs:
    for sequence in range(numSequences):
            try:
                os.makedirs(os.path.join(dataPath,sign,str(sequence)))
                os.makedirs(os.path.join(imgPath,sign,str(sequence)))
            except:
                pass
# if not os.path.exists("Sign_Data"):
#     os.mkdir("Sign_Data")
# else:
#     print("already exist")


In [5]:
os.path??

In [9]:
# while True:
#     inpt=input(print("Select which data are you collecting:\n0: Hello \n1: Thank you"))
#     if inpt == '0' or inpt == '1':
#         choice=signs[int(inpt)]
#         break
#     else:
#         print("Invalid input, please select again!!")
# choice   

In [10]:
vid=cv2.VideoCapture(0)
with mpHolistic.Holistic() as holistic:

    #taking label input 
     while True:
        blankImg = np.zeros(shape=[512, 512, 3], dtype=np.uint8)
        cv2.putText(blankImg,"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"0: Hello , 1: Thank You", (10,100),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"2: Hungry , 3: Food", (10,150),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"4: Hospital , 5: Washroom", (10,200),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"'ESC' to escape", (10,250),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('Select label',blankImg)
        inpt=cv2.waitKey(0)
        if inpt == 48 or inpt == 49 or inpt == 50 or inpt == 51 or inpt == 52 or inpt == 53:
            choice=signs[inpt-48]
            cv2.destroyWindow('Select label')
            break
        else:
            if inpt==27:
                break
            
    #checking camera is opened or not and taking data    
     while vid.isOpened() and inpt!=27 :
        for sequence in range(numSequences):
            for frameNum in range(sequenceLength+1):
                #checks for user input to close the windows                
                key=cv2.waitKey(1)                  
                success,img=vid.read()
                
                #checking if data is accessed or not from camera
                if not success:
                    break  
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                # img.flags.writeable=False
                results=holistic.process(img) 
                # img.flags.writeable=True     
                img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
                
                #draws landmarks
                drawLandmarks()
                
                #show feed for collecting datas and delays for 2 sec
                if frameNum == 0: 
                    cv2.putText(img, "Press 'ESC' to escape", (10,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.putText(img, 'press any key to conitnue', (10,60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 0), 1, cv2.LINE_AA)
                    cv2.imshow('Collecting Datas', img)
                    key= cv2.waitKey(0)
                    cv2.waitKey(1000)
                    
                #starts collecting datas    
                else: 
                    cv2.putText(img, f"Collecting Data for '{choice}' Video Number {sequence}", (15,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.imshow('Collecting Datas', img)
                    keypoints=extractKeypoints()  
                    
                    #saving flatten array to datapath
                    np.save(os.path.join(dataPath,choice,str(sequence),str(frameNum-1)),keypoints)
                    
                    #saving image to image path
                    jpgPath=os.path.join(imgPath,choice,str(sequence),str(frameNum-1))
                    cv2.imwrite(f"{jpgPath}.jpg",img)
                    key=2 #giving default value for key to avoid esc while taking data
                
                if key == 27 : #press esc to close the window
                    break     

            if key == 27 : #press esc to close the window
                break        
        if key == 27 : #press esc to close the window
            break             
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()            

# Preprocess Data and Create Labels and Features

In [40]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [41]:
label_map = {label:num for num, label in enumerate(signs)}

In [5]:
label_map

{'Ambulance': 0,
 'Fight': 1,
 'Clap': 2,
 'Hungry': 3,
 'Help': 4,
 'Medicine': 5,
 'Accident': 6,
 'ill': 7,
 'Hospital': 8,
 'Washroom': 9}

In [8]:
label_map[sign]

9

In [42]:
sequences, labels = [], []
for sign in signs:
    for sequence in range(numSequences):
        window = []
        for frameNum in range(sequenceLength):
            res = np.load(os.path.join(dataPath, sign, str(sequence), "{}.npy".format(frameNum)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[sign])

In [10]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [11]:
np.array(labels).shape

(900,)

In [12]:
np.array(sequences).shape

(900, 30, 1662)

In [17]:
arr=np.array([[[1,2],[3,4],[5,6]],
                 [[1,2],[3,4],[5,6]]])

In [84]:
arr.shape

NameError: name 'arr' is not defined

In [43]:
X = np.array(sequences)

In [53]:
X.shape

(900, 30, 1662)

In [44]:
y = to_categorical(labels).astype(int)

In [55]:
y.shape

(900, 10)

In [ ]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [25]:
y_test.shape

(9, 6)

# Save test datas and signs

In [46]:
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
try:
    os.makedirs(os.path.join(testDatasPath))
    os.makedirs(os.path.join(signsPath))
except:
    pass
np.save(os.path.join(testDatasPath,'x_test.npy'),X_test)
np.save(os.path.join(testDatasPath,'y_test.npy'),y_test)
np.save(os.path.join(signsPath,'signs.npy'),signs)

# Build and Train LSTM Neural Network

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

Build and Train model

In [74]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu', dropout=.2))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(10, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))


In [20]:
signs.shape[0]

10

In [21]:
res = [.7, 0.2, 0.1]

In [22]:
signs[np.argmax(res)]

'Ambulance'

In [75]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [76]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [77]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
26/26 [==============================] - 3s 66ms/step - loss: 2.4211 - categorical_accuracy: 0.1099
Epoch 2/2000
26/26 [==============================] - 2s 67ms/step - loss: 2.1633 - categorical_accuracy: 0.2012
Epoch 3/2000
26/26 [==============================] - 2s 66ms/step - loss: 1.9244 - categorical_accuracy: 0.2593
Epoch 4/2000
26/26 [==============================] - 2s 66ms/step - loss: 1.8341 - categorical_accuracy: 0.3074
Epoch 5/2000
26/26 [==============================] - 2s 66ms/step - loss: 1.4574 - categorical_accuracy: 0.4247
Epoch 6/2000
26/26 [==============================] - 2s 67ms/step - loss: 1.2730 - categorical_accuracy: 0.4827
Epoch 7/2000
26/26 [==============================] - 2s 66ms/step - loss: 1.1560 - categorical_accuracy: 0.5173
Epoch 8/2000
26/26 [==============================] - 2s 67ms/step - loss: 0.9994 - categorical_accuracy: 0.5914
Epoch 9/2000
26/26 [==============================] - 2s 61ms/step - loss: 0.8945 - categorical_

26/26 [==============================] - 2s 63ms/step - loss: 0.0957 - categorical_accuracy: 0.9704
Epoch 73/2000
26/26 [==============================] - 2s 58ms/step - loss: 0.1113 - categorical_accuracy: 0.9543
Epoch 74/2000
26/26 [==============================] - 2s 59ms/step - loss: 0.1062 - categorical_accuracy: 0.9617
Epoch 75/2000
26/26 [==============================] - 2s 59ms/step - loss: 0.0765 - categorical_accuracy: 0.9716
Epoch 76/2000
26/26 [==============================] - 2s 59ms/step - loss: 0.2020 - categorical_accuracy: 0.9136
Epoch 77/2000
26/26 [==============================] - 2s 60ms/step - loss: 0.1227 - categorical_accuracy: 0.9506
Epoch 78/2000
26/26 [==============================] - 2s 60ms/step - loss: 0.0787 - categorical_accuracy: 0.9728
Epoch 79/2000
26/26 [==============================] - 2s 59ms/step - loss: 0.1318 - categorical_accuracy: 0.9519
Epoch 80/2000
26/26 [==============================] - 2s 59ms/step - loss: 0.1449 - categorical_accur

26/26 [==============================] - 2s 59ms/step - loss: 0.0166 - categorical_accuracy: 0.9938
Epoch 144/2000
26/26 [==============================] - 2s 62ms/step - loss: 0.0145 - categorical_accuracy: 0.9951
Epoch 145/2000
26/26 [==============================] - 2s 60ms/step - loss: 0.0152 - categorical_accuracy: 0.9963
Epoch 146/2000
26/26 [==============================] - 2s 62ms/step - loss: 0.0468 - categorical_accuracy: 0.9827
Epoch 147/2000
26/26 [==============================] - 2s 64ms/step - loss: 0.0278 - categorical_accuracy: 0.9951
Epoch 148/2000
26/26 [==============================] - 2s 65ms/step - loss: 0.0273 - categorical_accuracy: 0.9926
Epoch 149/2000
26/26 [==============================] - 2s 68ms/step - loss: 0.0889 - categorical_accuracy: 0.9679
Epoch 150/2000
26/26 [==============================] - 2s 67ms/step - loss: 0.0998 - categorical_accuracy: 0.9679
Epoch 151/2000
26/26 [==============================] - 2s 64ms/step - loss: 0.0330 - categoric

26/26 [==============================] - 2s 72ms/step - loss: 0.0211 - categorical_accuracy: 0.9938
Epoch 215/2000
26/26 [==============================] - 2s 70ms/step - loss: 0.1646 - categorical_accuracy: 0.9506
Epoch 216/2000
26/26 [==============================] - 2s 75ms/step - loss: 0.0509 - categorical_accuracy: 0.9901
Epoch 217/2000
26/26 [==============================] - 2s 78ms/step - loss: 0.0194 - categorical_accuracy: 0.9951
Epoch 218/2000
26/26 [==============================] - 2s 67ms/step - loss: 0.0100 - categorical_accuracy: 0.9975
Epoch 219/2000
26/26 [==============================] - 2s 68ms/step - loss: 0.0108 - categorical_accuracy: 0.9988
Epoch 220/2000
26/26 [==============================] - 2s 68ms/step - loss: 0.0211 - categorical_accuracy: 0.9963
Epoch 221/2000
26/26 [==============================] - 2s 68ms/step - loss: 0.0151 - categorical_accuracy: 0.9938
Epoch 222/2000
26/26 [==============================] - 2s 70ms/step - loss: 0.0118 - categoric

KeyboardInterrupt: 

In [78]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_16 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_17 (LSTM)              (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 10)                650       
                                                                 
Total params: 590,986
Trainable params: 590,986
Non-trainable params: 0
_________________________________________________________________


# Make Prediction

In [79]:
#load signs, x_test, y_test
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))

In [80]:
res = model.predict(x_test)

In [81]:
signs[np.argmax(res[4])]

'Fight'

In [82]:
signs[np.argmax(y_test[4])]

'Fight'

In [ ]:
res.shape

(13, 6)

In [ ]:
y_test

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
res

In [83]:
np.argmax(res[0])

4

In [84]:
np.argmax(y_test[0])

4

# Save Model

In [85]:
model.save('SLGR_3.h5')

# Load Model

In [4]:
model.load_weights('SLR.h5')

NameError: name 'model' is not defined

# Evaluation using confusion matrix

In [62]:
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix, accuracy_score

In [63]:
true_labels=[]
for label in y_test:
    index=np.argmax(label)
    true_labels.append(signs[index])

In [64]:
true_labels

['Help',
 'Fight',
 'ill',
 'Washroom',
 'Fight',
 'Hungry',
 'Accident',
 'Help',
 'Hungry',
 'Fight',
 'Hungry',
 'Clap',
 'Fight',
 'Clap',
 'Hospital',
 'ill',
 'Help',
 'Washroom',
 'Hungry',
 'Ambulance',
 'ill',
 'Fight',
 'Clap',
 'Accident',
 'Washroom',
 'Accident',
 'Medicine',
 'Hungry',
 'ill',
 'Accident',
 'Medicine',
 'Clap',
 'ill',
 'Ambulance',
 'ill',
 'Ambulance',
 'Hospital',
 'Ambulance',
 'Ambulance',
 'Hospital',
 'Hospital',
 'Fight',
 'Ambulance',
 'Hungry',
 'ill',
 'Hospital',
 'Washroom',
 'Clap',
 'Medicine',
 'Clap',
 'Hungry',
 'Hungry',
 'ill',
 'Help',
 'Clap',
 'Accident',
 'Washroom',
 'Help',
 'ill',
 'Hungry',
 'Ambulance',
 'Washroom',
 'Fight',
 'Medicine',
 'Clap',
 'Help',
 'Medicine',
 'Hospital',
 'Ambulance',
 'Clap',
 'Fight',
 'Hospital',
 'Hungry',
 'Help',
 'ill',
 'Hungry',
 'Clap',
 'Ambulance',
 'Hungry',
 'Washroom',
 'Washroom',
 'ill',
 'ill',
 'Hospital',
 'ill',
 'Fight',
 'Hungry',
 'Help',
 'Help',
 'Fight']

In [65]:
y_predicted = model.predict(x_test)

In [66]:
predicted_labels=[]
for label in y_predicted:
    index=np.argmax(label)
    predicted_labels.append(signs[index])

In [67]:
predicted_labels

['Help',
 'Fight',
 'ill',
 'Washroom',
 'Fight',
 'Hungry',
 'Accident',
 'Help',
 'Hungry',
 'Fight',
 'Hungry',
 'Clap',
 'Fight',
 'Clap',
 'Hospital',
 'ill',
 'Help',
 'Washroom',
 'Hungry',
 'Help',
 'ill',
 'Fight',
 'Clap',
 'Accident',
 'Washroom',
 'Accident',
 'Medicine',
 'Hungry',
 'ill',
 'Fight',
 'Medicine',
 'Clap',
 'ill',
 'Ambulance',
 'ill',
 'Ambulance',
 'Hospital',
 'Ambulance',
 'Ambulance',
 'Hospital',
 'Hospital',
 'Fight',
 'Accident',
 'Washroom',
 'ill',
 'Hospital',
 'Washroom',
 'Clap',
 'Medicine',
 'Clap',
 'Hungry',
 'Hungry',
 'ill',
 'Help',
 'Clap',
 'Accident',
 'Washroom',
 'Help',
 'ill',
 'Hungry',
 'Ambulance',
 'Washroom',
 'Fight',
 'Medicine',
 'Clap',
 'Help',
 'Medicine',
 'Hospital',
 'Help',
 'Clap',
 'Fight',
 'Hospital',
 'Hungry',
 'Help',
 'ill',
 'Hungry',
 'Clap',
 'Ambulance',
 'Hungry',
 'Washroom',
 'Washroom',
 'ill',
 'ill',
 'Hospital',
 'ill',
 'Fight',
 'Hungry',
 'Help',
 'Help',
 'Fight']

In [68]:
y_true = np.argmax(y_test, axis=1).tolist()

In [69]:
y_predicted=np.argmax(y_predicted,axis=1).tolist()

In [55]:
y_true

[4, 2, 4, 0, 2, 1, 0, 3, 3]

In [ ]:
y_test

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
y_test.shape

(13, 6)

In [ ]:
y_predicted

[4, 5, 3, 3, 1, 4, 0, 0, 0, 1, 0, 2, 0]

In [ ]:
signs

array(['Hello', 'Thank You', 'Hungry', 'Food', 'Hospital', 'Washroom'],
      dtype='<U9')

In [56]:
multilabel_confusion_matrix(true_labels, predicted_labels,labels=signs)

array([[[7, 0],
        [0, 2]],

       [[6, 2],
        [0, 1]],

       [[7, 0],
        [1, 1]],

       [[7, 0],
        [1, 1]],

       [[6, 1],
        [1, 1]],

       [[9, 0],
        [0, 0]]], dtype=int64)

In [71]:
confusion_matrix(y_true, y_predicted)

array([[ 6,  0,  0,  0,  2,  0,  1,  0,  0,  0],
       [ 0, 10,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 12,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  9,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  5,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  4,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  8,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  8]], dtype=int64)

In [72]:
accuracy_score(y_true,y_predicted)

0.9444444444444444

In [ ]:
y_test.tolist()

[[0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0]]

In [70]:
list(zip(true_labels, predicted_labels))

[('Help', 'Help'),
 ('Fight', 'Fight'),
 ('ill', 'ill'),
 ('Washroom', 'Washroom'),
 ('Fight', 'Fight'),
 ('Hungry', 'Hungry'),
 ('Accident', 'Accident'),
 ('Help', 'Help'),
 ('Hungry', 'Hungry'),
 ('Fight', 'Fight'),
 ('Hungry', 'Hungry'),
 ('Clap', 'Clap'),
 ('Fight', 'Fight'),
 ('Clap', 'Clap'),
 ('Hospital', 'Hospital'),
 ('ill', 'ill'),
 ('Help', 'Help'),
 ('Washroom', 'Washroom'),
 ('Hungry', 'Hungry'),
 ('Ambulance', 'Help'),
 ('ill', 'ill'),
 ('Fight', 'Fight'),
 ('Clap', 'Clap'),
 ('Accident', 'Accident'),
 ('Washroom', 'Washroom'),
 ('Accident', 'Accident'),
 ('Medicine', 'Medicine'),
 ('Hungry', 'Hungry'),
 ('ill', 'ill'),
 ('Accident', 'Fight'),
 ('Medicine', 'Medicine'),
 ('Clap', 'Clap'),
 ('ill', 'ill'),
 ('Ambulance', 'Ambulance'),
 ('ill', 'ill'),
 ('Ambulance', 'Ambulance'),
 ('Hospital', 'Hospital'),
 ('Ambulance', 'Ambulance'),
 ('Ambulance', 'Ambulance'),
 ('Hospital', 'Hospital'),
 ('Hospital', 'Hospital'),
 ('Fight', 'Fight'),
 ('Ambulance', 'Accident'),
 ('Hungr

In [5]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import os,cv2
import mediapipe as mp
import time

mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils
vid=cv2.VideoCapture(0)
sequenceLength=30

#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))


#extract keypoints from image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

#load signs, x_test, y_test
# testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
# x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
# y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))



#load model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))  # 128
model.add(LSTM(128, return_sequences=True, activation='relu'))  # 64
model.add(LSTM(64, return_sequences=False, activation='relu'))  # 64
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))
model.load_weights('SLR.h5')
print(model.summary())
# real time testing
sequence=[]
sentence=[]
predictions = []
threshold=0.5
prediction_duration=[]
start_time=0.0
end_time=0.0
start_timer=True

dummy_arr=np.zeros([10,1662],dtype=float)

dummy_lst=[]

for e in dummy_arr:
    dummy_lst.append(list(e))

# ['Hello' 'Thank You' 'Hungry' 'Food' 'Hospital' 'Washroom']
def get_threshold(index):
    switcher = {
        0: 1.0, # for Hello
        1: 0.5,  # for Thank You
        2: 0.5,  # for Hungry
        3: 0.02,  # for Food
        4: 0.3,  # for Hospital
        5: 0.3   # for Washroom
    } 
    return switcher.get(index)


with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                     
    #checking camera is opened or not and taking data    
    while vid.isOpened():
        if start_timer:
            start_time=time.time()
        #checks for user input to close the windows                
        key=cv2.waitKey(1)                  
        success,img=vid.read()
        
        #checking if data is accessed or not from camera
        if not success:
            break  

        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        # img.flags.writeable=False
        results=holistic.process(img) 
        # img.flags.writeable=True     
        img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

        #draws landmarks
        drawLandmarks()

        #starts collecting datas    
        keypoints=extractKeypoints() 

        #appending to sequence list 
        sequence.append(keypoints)
        print(len(sequence))
        sequence = sequence[-30:]
        if len(sequence)==30:
            #prediction   
            # print(np.array(sequence).shape)
            # print(np.expand_dims(sequence, axis=0).shape)
            # temp_sequence=sequence
            # temp_sequence.append(dummy_lst)
            res=model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)
            predictions.append(np.argmax(res))
            
            #res=[1.0 if x>1 else x for x in res ] 
            # print(res)
            print(signs[np.argmax(res)])
            # threshold_val=get_threshold(np.argmax(res))
            if np.unique(predictions[-20:])[0]==np.argmax(res): 
                
                print(predictions[-20:])
                if res[np.argmax(res)] > threshold:
                     

                    if len(sentence) > 0: 
                        if signs[np.argmax(res)] != sentence[-1]:
                            sentence.append(signs[np.argmax(res)])
                            start_timer=True
                            end_time=time.time()
                            prediction_duration.append([signs[predictions[-1]],round(end_time-start_time,2)])

                        else:
                            start_timer=False    
                            
                    else:
                        end_time=time.time()
                        prediction_duration.append([signs[predictions[-1]],round(end_time-start_time,2)])
                        sentence.append(signs[np.argmax(res)])
                        start_timer=True
            else:
                start_timer=False
        if len(sentence) > 5: 
            sentence =[]
            # cv2.putText('Collecting Datas',"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA) 
            # cv2.imshow('Collecting Datas', img)
        
        cv2.rectangle(img, (0,0), (640, 40), (245, 117, 16), -1)     
        cv2.putText(img,' '.join(sentence), (10,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA) 
        cv2.imshow('SLR',img)
    
            
        if key == 27 : #press esc to close the window
            break   
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()

print(prediction_duration)

FileNotFoundError: [Errno 2] No such file or directory: 'Signs\\signs.npy'

In [1]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import os,cv2
import mediapipe as mp
import time

mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils
vid=cv2.VideoCapture(0)
sequenceLength=30

#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))


#extract keypoints from image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

#load signs, x_test, y_test
# testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
# x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
# y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))



#load model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))  # 128
model.add(LSTM(128, return_sequences=True, activation='relu'))  # 64
model.add(LSTM(64, return_sequences=False, activation='relu'))  # 64
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))
model.load_weights('SLR.h5')
print(model.summary())
# real time testing
sequence=[]
sentence=[]
predictions = []
threshold=0.5
prediction_duration=[]
start_time=0.0
end_time=0.0
start_timer=True

dummy_arr=np.zeros([10,1662],dtype=float)

dummy_lst=[]

for e in dummy_arr:
    dummy_lst.append(list(e))

# ['Hello' 'Thank You' 'Hungry' 'Food' 'Hospital' 'Washroom']
def get_threshold(index):
    switcher = {
        0: 1.0, # for Hello
        1: 0.5,  # for Thank You
        2: 0.5,  # for Hungry
        3: 0.02,  # for Food
        4: 0.3,  # for Hospital
        5: 0.3   # for Washroom
    } 
    return switcher.get(index)


with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                     
    #checking camera is opened or not and taking data    
    while vid.isOpened():
        if start_timer:
            start_time=time.time()
        #checks for user input to close the windows                
        key=cv2.waitKey(1)                  
        success,img=vid.read()
        
        #checking if data is accessed or not from camera
        if not success:
            break  

        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        # img.flags.writeable=False
        results=holistic.process(img) 
        # img.flags.writeable=True     
        img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

        #draws landmarks
        drawLandmarks()

        #starts collecting datas    
        keypoints=extractKeypoints() 

        #appending to sequence list 
        sequence.append(keypoints)
        print(len(sequence))
        sequence = sequence[-30:]
        if len(sequence)==30:
            #prediction   
            # print(np.array(sequence).shape)
            # print(np.expand_dims(sequence, axis=0).shape)
            # temp_sequence=sequence
            # temp_sequence.append(dummy_lst)
            res=model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)
            predictions.append(np.argmax(res))
            
            #res=[1.0 if x>1 else x for x in res ] 
            # print(res)
            print(signs[np.argmax(res)])
            # threshold_val=get_threshold(np.argmax(res))
            if np.unique(predictions[-20:])[0]==np.argmax(res): 
                
                print(predictions[-20:])
                if res[np.argmax(res)] > threshold:
                     

                    if len(sentence) > 0: 
                        if signs[np.argmax(res)] != sentence[-1]:
                            sentence.append(signs[np.argmax(res)])
                            start_timer=True
                            end_time=time.time()
                            prediction_duration.append([signs[predictions[-1]],round(end_time-start_time,2)])

                        else:
                            start_timer=False    
                            
                    else:
                        end_time=time.time()
                        prediction_duration.append([signs[predictions[-1]],round(end_time-start_time,2)])
                        sentence.append(signs[np.argmax(res)])
                        start_timer=True
            else:
                start_timer=False
        if len(sentence) > 5: 
            sentence =[]
            # cv2.putText('Collecting Datas',"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA) 
            # cv2.imshow('Collecting Datas', img)
        
        cv2.rectangle(img, (0,0), (640, 40), (245, 117, 16), -1)     
        cv2.putText(img,' '.join(sentence), (10,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA) 
        cv2.imshow('SLR',img)
    
            
        if key == 27 : #press esc to close the window
            break   
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()

print(prediction_duration)

FileNotFoundError: [Errno 2] No such file or directory: 'Signs\\signs.npy'